# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies


Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


In [2]:
# For some strange reason PIMC is crashing the second time it is called from Jupyter
models.pimc_use_declaring = False
models.pimc_use_defending = False

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding
1N Suggested bid from NN: CandidateBid(bid=1C  , insta_score=1.1999, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  , Explain=3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP))
1N is not in the bids from the neural network
PASS OK NN-value: 1.200
4H OK NN-value: 1.054
PASS OK NN-value: 0.868
4S OK NN-value: 1.198
PASS OK NN-value: 1.200
PASS OK NN-value: 1.165
PASS OK NN-value: 1.200
analyzing opening lead
C2
C2 OK
analyzing play
D3 ?? losing: 0.70
CA ?? losing: 1.02
C6 OK
D6 OK
DJ OK
DQ Forced
D5 OK
DA OK
D7 OK
DT OK
D4 OK
D8 OK
H6 OK
H2 OK
D9 OK
SQ OK
S5 OK
S2 OK
SK OK
H4 OK
HA OK
H7 OK
H9 OK
S4 OK
C3 Forced
SA ?? losing: 1.08
S9 Forced
S3 OK
C4 OK
ST Forced
H3 Forced
CK OK
C5 OK
HJ OK
C7 OK
C8 OK
CT OK
S6 Forced
CJ OK
S7 OK
H8 OK
C9 OK
D2 OK
S8 OK
H5 Forced
CQ OK
HT OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'who': 'Analysis',
 'quality': '1',
 'candidates': [{'call': '1C',
   'insta_score': 1.2,
   'alert': 'False',
   'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}],
 'hcp': [3.2, 4.0, 2.9, 2.7, 3.3, 3.9, 3.0, 2.7, 3.2, 3.9, 3.0, 2.8],
 'shape': [7.9, 8.9, 7.7],
 'explanation': '3+ !C -- 3+!C; 6-!D; 5-!H; 5-!S; 11-21 HCP'}

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'who': '',
 'quality': '0.8349',
 'hcp': [6.6, 2.1, 2.0, 2.2, 2.6, 3.5, 3.5, 3.5, 2.8, 3.5, 3.3, 3.2],
 'shape': [8.3, 7.3, 15.7],
 'candidates': [{'card': 'Cx',
   'insta_score': 0.287,
   'p_make_contract': 0.25,
   'expected_score_imp': 0.56,
   'msg': ''},
  {'card': 'Sx',
   'insta_score': 0.402,
   'p_make_contract': 0.25,
   'expected_score_imp': 0.55,
   'msg': ''},
  {'card': 'Hx',
   'insta_score': 0.209,
   'p_make_contract': 0.21,
   'expected_score_imp': -0.24,
   'msg': ''},
  {'card': 'DA',
   'insta_score': 0.061,
   'p_make_contract': 0.17,
   'expected_score_imp': -0.86,
   'msg': ''}],
 'samples': ['x.KTxx.AQxx.Txxx Jxxxxxx.A.xxx.Ax QTx.xxx.KJxx.Jxx AK.QJxxx.Tx.KQxx - 0.74219 | Sx:-650.0 Cx:-650.0 Hx:-650.0 DA:100.0',
  'x.KTxx.AQxx.Txxx QJTxxxx.Qx.xx.Kx Axx.Jxxx.Txx.Qxx Kx.Axx.KJxx.AJxx - 0.97248 | Sx:-620.0 Cx:-620.0 Hx:-650.0 DA:-650.0',
  'x.KTxx.AQxx.Txxx AKQxxxx.QJ.xx.Jx Txx.xxx.Txx.Qxxx Jx.Axxx.KJxx.AKx - 0.75425 | Sx:-650.0 Cx:-650.0 Hx:-650.0

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'S6',
 'who': 'IMP-calc',
 'quality': '0.8523',
 'candidates': [{'card': 'S6',
   'insta_score': 0.353,
   'expected_tricks_dd': 10.98,
   'p_make_contract': 0.96,
   'expected_score_imp': 1.15,
   'msg': ''},
  {'card': 'S2',
   'insta_score': 0.353,
   'expected_tricks_dd': 10.98,
   'p_make_contract': 0.96,
   'expected_score_imp': 1.15,
   'msg': ''},
  {'card': 'S3',
   'insta_score': 0.353,
   'expected_tricks_dd': 10.98,
   'p_make_contract': 0.96,
   'expected_score_imp': 1.15,
   'msg': ''},
  {'card': 'S7',
   'insta_score': 0.353,
   'expected_tricks_dd': 10.98,
   'p_make_contract': 0.96,
   'expected_score_imp': 1.15,
   'msg': ''},
  {'card': 'S8',
   'insta_score': 0.029,
   'expected_tricks_dd': 10.98,
   'p_make_contract': 0.96,
   'expected_score_imp': 1.15,
   'msg': ''},
  {'card': 'SJ',
   'insta_score': 0.003,
   'expected_tricks_dd': 10.89,
   'p_make_contract': 0.95,
   'expected_score_imp': 0.95,
   'msg': ''},
  {'card': 'D3',
   'insta_score': 0.29,


indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)
print(board)

Board(dealer='S', vuln=[False, True], hands=['9872.A3.KJ973.53', 'JAQ6.5286.5.J42Q', 'KT4.J.Q86.AKT976', '53.KQT974.AT42.8'], auction=['1C', '2H', 'X', '4H', '4S', 'PASS', 'PASS', 'PASS'], play=['DA', 'D3', 'D5', 'D6', 'C8', 'C3', 'CJ', 'CA', 'S4', 'S5', 'S8', 'SJ', 'H5', 'HJ', 'HQ', 'HA', 'S2', 'SA', 'ST', 'S3', 'H2', 'SK', 'H4', 'H3', 'D8', 'D2', 'DJ', 'S6', 'SQ', 'C6', 'H7', 'S7', 'H8', 'C7', 'HK', 'S9', 'C5', 'C2', 'CT', 'HT', 'CK', 'H9', 'D7', 'C4', 'DQ', 'D4', 'DK', 'H6', 'D9', 'CQ', 'C9', 'DT'])


In [12]:
card_by_card = CardByCard(*board, models, sampler, False)


In [13]:
await card_by_card.analyze()

analyzing the bidding
1C OK NN-value: 1.200
2H OK NN-value: 1.148
X Suggested bid from NN: CandidateBid(bid=PASS, insta_score=1.1132, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  , Explain= -- ; 13- HCP))
X NN-values:CandidateBid(bid=X   , insta_score=0.0836, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  )
4H Suggested bid from NN: CandidateBid(bid=3H  , insta_score=0.6403, expected_score=  136, expected_mp=---, expected_imp= -0.1, expected_tricks= 9.19, adjust=1.54, alert=  , Explain=Preemptive -- 2+!H; 3-8 HCP))
4H NN-values:CandidateBid(bid=4H  , insta_score=0.4117, expected_score=  106, expected_mp=---, expected_imp= -1.1, expected_tricks= 9.31, adjust=0.99, alert=  )
4S Suggested bid from NN: CandidateBid(bid=PASS, insta_score=1.0817, expected_score=  146, expected_mp=---, expected_imp=---, expected_tricks= 7.94, adjust=0.00, alert=  , Explain= -- 3+!C; 6-!D; 5-!H; 5-!S; 11-16 HCP)

the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [14]:
card_by_card.cards['S4'].to_dict()

{'card': 'HJ',
 'who': 'IMP-calc',
 'quality': '0.4812',
 'hcp': [7.5, 8.1],
 'shape': [3.1, 4.2, 3.2, 2.4, 2.4, 6.1, 2.5, 1.9],
 'candidates': [{'card': 'HJ',
   'insta_score': 0.29,
   'expected_tricks_dd': 7.54,
   'p_make_contract': 0.0,
   'expected_score_imp': 1.16,
   'msg': ''},
  {'card': 'DQ',
   'insta_score': 0.254,
   'expected_tricks_dd': 7.47,
   'p_make_contract': 0.01,
   'expected_score_imp': 1.05,
   'msg': ''},
  {'card': 'D8',
   'insta_score': 0.099,
   'expected_tricks_dd': 7.47,
   'p_make_contract': 0.01,
   'expected_score_imp': 1.05,
   'msg': ''},
  {'card': 'CK',
   'insta_score': 0.144,
   'expected_tricks_dd': 7.41,
   'p_make_contract': 0.0,
   'expected_score_imp': 0.92,
   'msg': ''},
  {'card': 'S4',
   'insta_score': 0.043,
   'expected_tricks_dd': 7.14,
   'p_make_contract': 0.01,
   'expected_score_imp': 0.41,
   'msg': ''},
  {'card': 'ST',
   'insta_score': 0.006,
   'expected_tricks_dd': 7.13,
   'p_make_contract': 0.0,
   'expected_score_imp': 

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [15]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'who': '',
 'quality': '0.7976',
 'hcp': [4.2, 2.2, 3.6, 3.0, 2.8, 4.2, 3.2, 2.7, 4.1, 1.0, 2.2, 5.7],
 'shape': [11.1, 6.4, 13.4],
 'candidates': [{'card': 'DA',
   'insta_score': 0.042,
   'p_make_contract': 0.1,
   'expected_score_imp': 0.74,
   'msg': ''},
  {'card': 'C8',
   'insta_score': 0.365,
   'p_make_contract': 0.1,
   'expected_score_imp': 0.39,
   'msg': ''},
  {'card': 'HK',
   'insta_score': 0.523,
   'p_make_contract': 0.06,
   'expected_score_imp': 0.21,
   'msg': 'suit adjust=0.5'},
  {'card': 'Sx',
   'insta_score': 0.039,
   'p_make_contract': 0.03,
   'expected_score_imp': -0.84,
   'msg': ''}],
 'samples': ['xx.KQTxxx.ATxx.x KQxx.x.Qxxxx.AQx Axx.Jxxxx.Jx.Jxx JTxx.A.Kx.KTxxxx - 0.73866 | HK:-450.0 C8:-420.0 DA:-420.0 Sx:-450.0',
  'xx.KQTxxx.ATxx.x KTxx.Ax.QJx.xxxx xx.Jxxxx.Kxxx.Tx AQJxx..xx.AKQJxx - 0.88019 | HK:-480.0 C8:-480.0 DA:-450.0 Sx:-480.0',
  'xx.KQTxxx.ATxx.x Txxx.x.Qxxx.AKxx Axx.AJxxx.xxx.xx KQJx.x.KJ.QJTxxx - 0.90147 | HK:50.0 C8:50.0